In [3]:
import pandas as pd
import sys
import csv
from datetime import datetime, timedelta
from collections import OrderedDict
import numpy as np
import time
import pyblp
import auxiliary as aux
import sqldf
import pysqldf as ps
from pandasql import sqldf
import pandasql
import os
import re
import itertools
import shutil

def int_to_month(value):
        year = np.floor((value - 1) / 12)
        month = value - 12 * year
        return year, month

def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a

# parsing info.txt file, returns dictionary of info
def parse_info(code):
    file = open('../../../../All/m_' + code + '/info.txt', mode = 'r')
    info_file = file.read()
    file.close()

    all_info_elements = re.finditer('\[(.*?):(.*?)\]', info_file, re.DOTALL)
    info_dict = {}
    for info in all_info_elements:
        info_name = info.group(1).strip()
        info_content = info.group(2).strip()
        info_dict[info_name] = info_content
    return info_dict

# getting the parties from the info dictionary, but you must use
# get_parties(info_dict["MergingParties"]) so info_str = info_dict["MergingParties"]
def get_parties(info_str):
    all_parties = re.finditer('{(.*?)}', info_str, re.DOTALL)
    merging_parties = []
    for i in all_parties:
        merging_parties.append(i.group(1).strip())
    return merging_parties

def get_date_range(initial_year_string, final_year_string, pre_months = 24, post_months = 24):
        initial_dt = datetime.strptime(initial_year_string, '%Y-%m-%d')
        final_dt = datetime.strptime(final_year_string, '%Y-%m-%d')
        initial_month_int = initial_dt.year * 12 + initial_dt.month
        final_month_int = final_dt.year * 12 + final_dt.month
        min_year, min_month = int_to_month(initial_month_int - pre_months)
        max_year, max_month = int_to_month(final_month_int + post_months)

        string_init = str(int(min_year)) + "-" + str(int(min_month))
        string_final = str(int(max_year)) + "-" + str(int(max_month))
        years_range = pd.date_range(string_init, string_final, freq='MS').strftime("%Y").tolist()
        months_range = pd.date_range(string_init, string_final, freq='MS').strftime("%m").tolist()

        date_range = pd.DataFrame(zip(years_range, months_range))

        return date_range

#getting the owners from aux
def append_owners(code, df, month_or_quarter,add_dhhi = False):
# # Load list of UPCs and brands
    upcs = pd.read_csv('../../../../All/m_' + code + '/intermediate/upcs.csv', delimiter = ',', index_col = 'upc')
    upcs = upcs['brand_code_uc']
    upc_map = upcs.to_dict()

# # Map brands to dataframe (by UPC)
    df['brand_code_uc'] = df['upc'].map(upc_map)

# # Load ownership assignments
    brand_to_owner = pd.read_csv('../../../../All/m_' + code + '/properties/ownership.csv', delimiter = ',', index_col = 'brand_code_uc')

# # Assign min/max year and month when listed as zero in ownership mapping
    min_year = df['year'].min()
    max_year = df['year'].max()

    if month_or_quarter == 'month':
        min_month = df.loc[df['year']==min_year,'month'].min()
        max_month = df.loc[df['year']==max_year,'month'].max()
    elif month_or_quarter == 'quarter':
        min_month = (3*(df.loc[df['year']==min_year,'quarter']-1)+1).min()
        max_month = (3*df.loc[df['year']==max_year,'quarter']).max()

    # Remove Onwership that starts later than the latest time in the dataframe
    brand_to_owner = brand_to_owner[(brand_to_owner['start_year'] < max_year) | ((brand_to_owner['start_year'] == max_year)&(brand_to_owner['start_month'] <= max_month))]
    # Remove Onwership that ends earlier than the earliest time in the dataframe
    brand_to_owner = brand_to_owner[(brand_to_owner['end_year'] > min_year) | ((brand_to_owner['end_year'] == min_year)&(brand_to_owner['end_month'] >= min_month)) | (brand_to_owner['end_year'] == 0)]

    brand_to_owner.loc[(brand_to_owner['start_month']==0) | (brand_to_owner['start_year']<min_year) | ((brand_to_owner['start_year']==min_year)&(brand_to_owner['start_month']<min_month)),'start_month'] = min_month
    brand_to_owner.loc[(brand_to_owner['start_year']==0) | (brand_to_owner['start_year']<min_year),'start_year'] = min_year
    brand_to_owner.loc[(brand_to_owner['end_month']==0) | (brand_to_owner['end_year']>max_year) | ((brand_to_owner['end_year']==max_year)&(brand_to_owner['end_month']>max_month)),'end_month'] = max_month
    brand_to_owner.loc[(brand_to_owner['end_year']==0) | (brand_to_owner['end_year']>max_year),'end_year'] = max_year

    # Throw error if (1) dates don't span the entirety of the sample period or
    # (2) ownership dates overlap
    brand_to_owner_test = brand_to_owner.copy()
    brand_to_owner_test = brand_to_owner_test.sort_values(by=['brand_code_uc', 'start_year', 'start_month'])
    
    if month_or_quarter == 'month':
        min_date = pd.to_datetime(dict(year=df.year, month=df.month, day=1)).min()
        max_date = pd.to_datetime(dict(year=df.year, month=df.month, day=1)).max()
        brand_to_owner_test['start_date_test'] = pd.to_datetime(dict(year=brand_to_owner_test.start_year, month=brand_to_owner_test.start_month, day=1))
        brand_to_owner_test['end_date_test'] = pd.to_datetime(dict(year=brand_to_owner_test.end_year, month=brand_to_owner_test.end_month, day=1))
    elif month_or_quarter == 'quarter':
        min_date = pd.to_datetime(dict(year=df.year, month=3*(df.quarter-1)+1, day=1)).min()
        max_date = pd.to_datetime(dict(year=df.year, month=3*df.quarter, day=1)).max()
        brand_to_owner_test.loc[:,'start_month'] = 3*(np.ceil(brand_to_owner_test['start_month']/3)-1)+1
        brand_to_owner_test.loc[:,'end_year'] = np.where(3*(np.floor(brand_to_owner_test.end_month/3)) > 0, brand_to_owner_test.end_year, brand_to_owner_test.end_year - 1)
        brand_to_owner_test.loc[:,'end_month'] = np.where(3*(np.floor(brand_to_owner_test.end_month/3)) > 0, 3*(np.floor(brand_to_owner_test.end_month/3)), 12)
        brand_to_owner_test['start_date_test'] = pd.to_datetime(dict(year=brand_to_owner_test.start_year, month=brand_to_owner_test.start_month, day=1))
        brand_to_owner_test['end_date_test'] = pd.to_datetime(dict(year=brand_to_owner_test.end_year, month=brand_to_owner_test.end_month, day=1))

    brand_dates = brand_to_owner_test.groupby('brand_code_uc')[['start_date_test', 'end_date_test']].agg(['min', 'max'])
    if ((brand_dates.start_date_test['min']!=min_date).sum() + (brand_dates.end_date_test['max']!=max_date).sum() > 0):
        print('Ownership definitions does not span the entire sample period:')
        for index, row in brand_dates.iterrows():
            if row.start_date_test['min'] != min_date or row.end_date_test['max'] != max_date:
                print(index)
                print('start_date: ', row.start_date_test['min'])
                print('end_date: ', row.end_date_test['max'])

    brand_to_owner_test['owner_num'] = brand_to_owner_test.groupby('brand_code_uc').cumcount()+1
    max_num_owner = brand_to_owner_test['owner_num'].max()
    brand_to_owner_test = brand_to_owner_test.set_index('owner_num',append=True)
    brand_to_owner_test = brand_to_owner_test.unstack('owner_num')
    brand_to_owner_test.columns = ['{}_{}'.format(var, num) for var, num in brand_to_owner_test.columns]

    for ii in range(2,max_num_owner+1):
        overlap_or_gap = (brand_to_owner_test['start_year_' + str(ii)] < brand_to_owner_test['end_year_' + str(ii-1)]) | \
            ((brand_to_owner_test['start_year_' + str(ii)] == brand_to_owner_test['end_year_' + str(ii-1)]) & \
            (brand_to_owner_test['start_month_' + str(ii)] != (brand_to_owner_test['end_month_' + str(ii-1)] + 1))) | \
            ((brand_to_owner_test['start_year_' + str(ii)] > brand_to_owner_test['end_year_' + str(ii-1)]) & \
            ((brand_to_owner_test['start_month_' + str(ii)] != 1) | (brand_to_owner_test['end_month_' + str(ii-1)] != 12)))
        if overlap_or_gap.sum() > 0:
            brand_to_owner_test['overlap'] = overlap_or_gap
            indices = brand_to_owner_test[brand_to_owner_test['overlap'] != 0].index.tolist()
            for index in indices:
                print(brand_to_owner_test.loc[index])
            raise Exception('There are gaps or overlap in the ownership mapping.')

    # Merge on brand and date intervals
    if month_or_quarter == 'month':
        brand_to_owner['start_date'] = pd.to_datetime(dict(year=brand_to_owner.start_year, month=brand_to_owner.start_month, day=1))
        brand_to_owner['end_date'] = pd.to_datetime(dict(year=brand_to_owner.end_year, month=brand_to_owner.end_month, day=1))
        df['date'] = pd.to_datetime(dict(year=df.year, month=df.month, day=1))
        if add_dhhi:
            sqlcode = '''
            select df.upc, df.year, df.month, df.shares, df.dma_code, df.brand_code_uc, brand_to_owner.owner
            from df
            inner join brand_to_owner on df.brand_code_uc=brand_to_owner.brand_code_uc AND df.date >= brand_to_owner.start_date AND df.date <= brand_to_owner.end_date
            '''
        else:
            sqlcode = '''
            select df.upc, df.year, df.month, df.prices, df.shares, df.volume, df.dma_code, df.brand_code_uc, df.sales, brand_to_owner.owner
            from df
            inner join brand_to_owner on df.brand_code_uc=brand_to_owner.brand_code_uc AND df.date >= brand_to_owner.start_date AND df.date <= brand_to_owner.end_date
            '''
    elif month_or_quarter == 'quarter':
        brand_to_owner.loc[:,'start_month'] = 3*(np.ceil(brand_to_owner['start_month']/3)-1)+1
        brand_to_owner.loc[:,'end_year'] = np.where(3*(np.floor(brand_to_owner.end_month/3)) > 0, brand_to_owner.end_year, brand_to_owner.end_year - 1)
        brand_to_owner.loc[:,'end_month'] = np.where(3*(np.floor(brand_to_owner.end_month/3)) > 0, 3*(np.floor(brand_to_owner.end_month/3)), 12)
        brand_to_owner['start_date'] = pd.to_datetime(dict(year=brand_to_owner.start_year, month=brand_to_owner.start_month, day=1))
        brand_to_owner['end_date'] = pd.to_datetime(dict(year=brand_to_owner.end_year, month=brand_to_owner.end_month, day=1))
        df['date'] = pd.to_datetime(dict(year=df.year, month=3*(df.quarter-1)+1, day=1))
        if add_dhhi:
            sqlcode = """
            select 
                df.upc, df.year, df.quarter, df.shares, df.dma_code, df.brand_code_uc, brand_to_owner.owner
            from 
                df
            inner join 
                brand_to_owner 
                    on df.brand_code_uc=brand_to_owner.brand_code_uc AND df.date >= brand_to_owner.start_date AND df.date <= brand_to_owner.end_date
            """
        else:
            sqlcode = """
            select 
                df.upc, df.year, df.quarter, df.prices, df.shares, df.volume, df.dma_code, df.brand_code_uc, df.sales, brand_to_owner.owner
            from 
                df
            inner join 
                brand_to_owner 
                    on df.brand_code_uc=brand_to_owner.brand_code_uc AND df.date >= brand_to_owner.start_date AND df.date <= brand_to_owner.end_date
            """
    df_own = sqldf(sqlcode,locals())

    return df_own


In [27]:
def table_1(code):
    
    # must have 4 ../../../.. because i'm inside a folder inside Main
    # opening data_month file
    df = (pd.read_csv('../../../../All/m_' + code + '/intermediate/data_month.csv'))
    
    ### Part 1: making df complete with year, months and all dma_codes exhaustive
    ### note - df_own does NOT have all dates, only dates which the upc-year-month sales > 0!!
    
    # create list of all year month combinations
    info_dict = parse_info(code)
    date_range = get_date_range(info_dict['DateAnnounced'], info_dict['DateCompleted'], pre_months = 24, post_months = 24)
    
    # calculate number of rows there should be
    upcs = (pd.read_csv('../../../../All/m_' + code + '/intermediate/upcs.csv', delimiter = ','))['upc']
    #total_rows = len(upcs)*len(date_range)
    #total_columns = len(pivoted.columns)
    
    # take upc-year-month-dma combinations now
    repeated_upcs = (pd.concat([upcs]*len(date_range))).sort_values()
    repeated_upcs.columns = ['upc']
    repeated_upcs = repeated_upcs.reset_index(drop=True)

    repeated_dates = pd.concat([date_range]*len(upcs))
    repeated_dates.columns = ['year', 'month']
    repeated_dates = repeated_dates.reset_index(drop=True)
    
    unique_dma_codes = pd.DataFrame(df['dma_code'].unique())
    repeated_dmas = pd.concat([unique_dma_codes]*len(upcs)*len(date_range))
    repeated_dmas.columns = ['dma_code']
    repeated_dmas = repeated_dmas.reset_index(drop=True)

    # creating the empty dataframe with all upc-year-month-dma combinations
    empty_to_merge = pd.concat([pd.concat([repeated_upcs, repeated_dates], axis= 1)]*len(unique_dma_codes))
    empty_to_merge = empty_to_merge.reset_index(drop=True)
    empty_to_merge = empty_to_merge.sort_values(by = ['upc', 'year', 'month'])
    empty_to_merge.insert(1, 'dma_code', repeated_dmas)
    empty_to_merge = empty_to_merge.apply(pd.to_numeric)
    
    #df_full = pd.merge(df, empty_to_merge)
    # actual merging with incomplete df, filling in 0's for all spots where no sales
    empty_to_merge_dma_full = pd.merge(df, empty_to_merge, how = "right", on = ['upc', 'dma_code','year', 'month'])
    empty_to_merge_dma_full.fillna(0, inplace=True)
    empty_to_merge_dma_full = empty_to_merge_dma_full.apply(pd.to_numeric)

    empty_to_merge_dma_full['sold_in_usa'] = 0
    empty_to_merge_dma_full.loc[empty_to_merge_dma_full.sales != 0, 'sold_in_usa'] = 1
    
    ### Part 2: extracting ownership using full df with all upc-year-month-dma combinations
    df_own = append_owners(code, empty_to_merge_dma_full, 'month')
    df_own['sold_in_usa'] = 0
    df_own.loc[empty_to_merge_dma_full.sales != 0, 'sold_in_usa'] = 1
    
    # extract merging parties
    merging_parties = get_parties(info_dict["MergingParties"])
    
    # extracting year and month of date completed
    year = int((info_dict['DateCompleted'])[:4])
    month = int((info_dict['DateCompleted'])[5:7])
    
    
    df_own.to_csv('try_this.csv')
    
    # pivoting the dmas and having sales and volume for each upc year month
    pivoted = df_own.pivot_table(index = ['upc','year','month','owner'], columns = 'dma_code', values = ['volume','sales']).reset_index()

    # filling in for 0
    pivoted.fillna(0, inplace=True)
    
    ### Part 3: Additional info
    
    #Total sales of the product in that quarter across the entire US.  Do volume and dollar sales.  For UPCs that aren't sold, it will be 0.
    pivoted.loc[:,'total_sales'] = pivoted[['sales']].sum(axis=1)
    pivoted.loc[:,'total_volume'] = pivoted[['volume']].sum(axis=1)
    
    # dummy for whether product is sold in that month or not in the US
    pivoted['sold_in_usa'] = 0
    pivoted.loc[pivoted.total_sales != 0, 'sold_in_usa'] = 1
    
    # dummies for whether the product is involved in a merger and whether it's post-merger
    # create column of zeroes
    pivoted['merging_party'] = 0
    pivoted['post_merger'] = 0
    
    #assign 1's if owner = merging parties
    pivoted.loc[pivoted['owner'].isin(merging_parties), 'merging_party'] = 1
    
    # setting = 1 if month and year are greater than date completed for the merging parties
    pivoted.loc[(pivoted['merging_party'] == 1) & (pivoted['year'] >= year) & (pivoted['month'] >= month), 'post_merger'] = 1
    
    pivoted.loc[(pivoted['merging_party'] == 1) & (pivoted['year'] > year), 'post_merger'] = 1
    pivoted.loc[(pivoted['merging_party'] == 1) & (pivoted['year'] == year) & (pivoted['month'] >= month), 'post_merger'] = 1
    
    # export to csv
    #pivoted.to_csv('m_' + code +'/pivoted_data.csv', sep = ',', encoding = 'utf-8')
    
    log_out = open('try_1.log', 'w')
    log_err = open('try_1.err', 'w')
    sys.stdout = log_out
    sys.stderr = log_err
    
    return pivoted, empty_to_merge_dma_full


In [28]:
code = '1924129020_1'
#os.mkdir('m_' + code)
table1, empty_to_merge_dma_full = table_1(code)


empty_to_merge_dma_full

,upc,dma_code,year,month,prices,shares,sales,volume,sold_in_usa,brand_code_uc,date
0,2150000064,500,2005,11,0.000000,0.000000,0.000000,0.000000,0,586519,2005-11-01
1,2150000064,606,2005,11,0.000000,0.000000,0.000000,0.000000,0,586519,2005-11-01
2,2150000064,825,2005,11,0.000000,0.000000,0.000000,0.000000,0,586519,2005-11-01
3,2150000064,637,2005,11,0.000000,0.000000,0.000000,0.000000,0,586519,2005-11-01
4,2150000064,543,2005,11,0.000000,0.000000,0.000000,0.000000,0,586519,2005-11-01
...,...,...,...,...,...,...,...,...,...,...,...
211579,947582845316,512,2010,8,8.709093,0.203969,400.963059,46.039588,1,536746,2010-08-01
211580,947582845316,628,2010,8,0.000000,0.000000,0.000000,0.000000,0,536746,2010-08-01
211581,947582845316,881,2010,8,0.000000,0.000000,0.000000,0.000000,0,536746,2010-08-01
211582,947582845316,675,2010,8,0.000000,0.000000,0.000000,0.000000,0,536746,2010-08-01


In [24]:
#df = (pd.read_csv('../../../../All/m_' + code + '/intermediate/data_month.csv'))

empty_to_merge_dma_full.to_csv('empty_merged_df.csv')

In [70]:
code = '2641303020_8'
#table_1(code)

# date range, upcs, df
info_dict = parse_info(code)
date_range = get_date_range(info_dict['DateAnnounced'], info_dict['DateCompleted'], pre_months = 24, post_months = 24)

# calculate number of rows there should be
upcs = (pd.read_csv('../../../../All/m_' + code + '/intermediate/upcs.csv', delimiter = ','))['upc']

df = (pd.read_csv('../../../../All/m_' + code + '/intermediate/data_month.csv'))


# take upc-year-month-dma combinations now
repeated_upcs = (pd.concat([upcs]*len(date_range))).sort_values()
repeated_upcs.columns = ['upc']
repeated_upcs = repeated_upcs.reset_index(drop=True)

repeated_dates = pd.concat([date_range]*len(upcs))
repeated_dates.columns = ['year', 'month']
repeated_dates = repeated_dates.reset_index(drop=True)

unique_dma_codes = pd.DataFrame(df['dma_code'].unique())
repeated_dmas = pd.concat([unique_dma_codes]*len(upcs)*len(date_range))
repeated_dmas.columns = ['dma_code']
repeated_dmas = repeated_dmas.reset_index(drop=True)

# creating the empty dataframe with all upc-year-month-dma combinations
empty_to_merge = pd.concat([pd.concat([repeated_upcs, repeated_dates], axis= 1)]*len(unique_dma_codes))
empty_to_merge = empty_to_merge.reset_index(drop=True)
empty_to_merge = empty_to_merge.sort_values(by = ['upc', 'year', 'month'])
empty_to_merge.insert(1, 'dma_code', repeated_dmas)
empty_to_merge = empty_to_merge.apply(pd.to_numeric)

In [72]:
empty_to_merge.to_csv('check_empty_to_merge.csv')

In [991]:
#datetime.strptime((str(pivoted['month']) +' ' + str(pivoted['year'])), "%m %Y")

 # make a date time from the pivoted table
    # make a date time from datecompleted
    # compare the two
    date_completed = datetime.strptime((info_dict['DateCompleted']), '%Y-%m-%d')

(datetime.strptime(str(pivoted['month']) +' ' + str(pivoted['year']), "%m %Y") >= date_completed), 'post_merger'
date = datetime.strptime(str(pivoted['month'][1]) +' ' + str(pivoted['year'][1]), "%m %Y")
date_completed

code = '2641303020_8'

pivoted = vivek_table(code)
#pivoted.to_datetime()

#str(pivoted['month'][1]) +' ' +str(pivoted['year'][1])

df2 = pd.to_datetime(pivoted['month'])

dftry = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                    'day': [1, 3],
                      'seconds': [5, 13]})
pd.to_datetime(dftry[['year','month','day']])
#pd.to_datetime(pivoted[['year','month']])
pivoted.columns


MultiIndex([(          'upc',    ''),
            (         'year',    ''),
            (        'month',    ''),
            (        'owner',    ''),
            (        'sales', 500.0),
            (        'sales', 501.0),
            (        'sales', 503.0),
            (        'sales', 504.0),
            (        'sales', 506.0),
            (        'sales', 507.0),
            ...
            (       'volume', 855.0),
            (       'volume', 862.0),
            (       'volume', 866.0),
            (       'volume', 868.0),
            (       'volume', 881.0),
            (  'total_sales',    ''),
            ( 'total_volume',    ''),
            (  'sold_in_usa',    ''),
            ('merging_party',    ''),
            (  'post_merger',    '')],
           names=[None, 'dma_code'], length=227)

In [902]:
# this is changing the original df from data_month so that it has all the upc-year-month-dma combinations 
# so that the df that is used for the append_owners is complete 

code = '2641303020_8'
info_dict = parse_info(code)
date_range = get_date_range(info_dict['DateAnnounced'], info_dict['DateCompleted'], pre_months = 24, post_months = 24)
   
df = (pd.read_csv('../../../../All/m_' + code + '/intermediate/data_month.csv'))
upcs = (pd.read_csv('../../../../All/m_' + code + '/intermediate/upcs.csv', delimiter = ','))['upc']

# take upc-year-month-dma combinations now
repeated_upcs = (pd.concat([upcs]*len(date_range))).sort_values(ignore_index = True)
repeated_upcs.columns = ['upc']

repeated_dates = pd.concat([date_range]*len(upcs), ignore_index = True, names = ['year', 'month'])
repeated_dates.columns = ['year', 'month']

unique_dma_codes = pd.DataFrame(df['dma_code'].unique())
repeated_dmas = pd.concat([unique_dma_codes]*len(upcs)*len(date_range), ignore_index = True, names = ['dma_code'])
repeated_dmas.columns = ['dma_code']

empty_to_merge = pd.concat([pd.concat([repeated_upcs, repeated_dates], axis= 1)]*len(unique_dma_codes))
empty_to_merge = empty_to_merge.sort_values(by = ['upc', 'year', 'month'], ignore_index =True)
empty_to_merge.insert(1, 'dma_code', repeated_dmas)
empty_to_merge = empty_to_merge.apply(pd.to_numeric)

# actual merging
empty_to_merge_dma_full = pd.merge(df, empty_to_merge, how = "right", on = ['upc', 'dma_code','year', 'month'])
empty_to_merge_dma_full.fillna(0, inplace=True)

empty_to_merge_dma_full = empty_to_merge_dma_full.apply(pd.to_numeric)


# want upc - dma_code - year - month

#empty_to_merge.to_csv('pre_merging.csv')
empty_to_merge_dma_full.to_csv('major_full_dma_everything.csv')
#df.to_csv('df_to_check.csv')



In [ ]:
## filling in for missing owners other tries


#pivoted_withoutownership = df.pivot_table(index = ['upc','year','month'], columns = 'dma_code', values = ['volume','sales']).reset_index()

#pivoted_withoutownership

# creating empty dataframe with all the upc-year-month combinations needed
empty_to_merge = pd.concat([repeated_upcs, repeated_dates], axis= 1)
empty_to_merge = empty_to_merge.apply(pd.to_numeric)
    
# merge the pivoted and the full upc-year-month set
#pivoted_full = pd.merge(pivoted, empty_to_merge, how = "right", on = ['upc', 'year', 'month'])

# fill in for zeros in no sales upc-year-month combinations
pivoted_full.fillna(0, inplace=True)
    
# add row of zeroes for all that are missing
repeated_upcs = (pd.concat([upcs]*len(date_range))).sort_values(ignore_index = True)
repeated_upcs.columns = ['upc']

repeated_dates = pd.concat([date_range]*len(upcs), ignore_index = True, names = ['year', 'month'])
repeated_dates.columns = ['year', 'month']

unique_dma_codes = pd.DataFrame(df['dma_code'].unique())
repeated_dmas = pd.concat([unique_dma_codes]*len(upcs)*len(date_range), ignore_index = True, names = ['dma_code'])
repeated_dmas.columns = ['dma_code']

# fill in for missing owners
# first extract all the upc-year-month-owner possibilities
upc_brand_entire_dates = df_own[['upc','brand_code_uc', 'owner']].set_index('brand_code_uc')
upc_brand_entire_dates = upc_brand_entire_dates.drop_duplicates()
upc_brand_entire_dates = pd.merge(upc_brand_entire_dates, brand_to_owner[['start_month', 'start_year', 'end_month', 'end_year']], right_index = True, left_index = True)

# make a check statement or print statement here to indicate that the upc_brand_entire_dates ACTUALLY MATCHES date_range
missing_owners = pivoted_full.loc[try_1['owner'] == 0, ['upc', 'year', 'month']]

In [76]:
codes = ['1924129020_1', '2641303020_8', '2823116020_9']

for code in codes:
    
    #if os.path.exists('m_' + code):
     #   shutil.rmtree('m_' + code)
      #  table_1(code)
    
    os.mkdir('m_' + code)
    table_1(code)

In [782]:
code = '2641303020_8'
df = (pd.read_csv('../../../../All/m_' + code + '/intermediate/data_month.csv'))
df_own, brand_to_owner = append_owners(code, df, 'month',add_dhhi = False)

try_1, empty_to_merge = vivek_table(code)

upc_brand_entire_dates = df_own[['upc','brand_code_uc', 'owner']].set_index('brand_code_uc')
upc_brand_entire_dates = upc_brand_entire_dates.drop_duplicates()

len(upc_brand_entire_dates)
# find matching brand_code_ucs, paste in start date and end date

upc_brand_entire_dates = pd.merge(upc_brand_entire_dates, brand_to_owner[['start_month', 'start_year', 'end_month', 'end_year']], right_index = True, left_index = True)


upc_brand_entire_dates

#try_1.loc[try_1['owner'] = 0, 'owner'] = 


# check the upc in upc_brand_entire_dates (upc in both tables)

# check > start_month start_year & < end_month_end_year (month year in try_1, start_month...

# paste try_1.loc[try_1['owner'] = 0, 'owner'] = that owner


#try_1.where(try_1['owner'] != 0, 
 #          if )

# extracts upcs which have owner == 0
missing_owners = try_1.loc[try_1['owner'] == 0, ['upc', 'year', 'month']]

#try_1.loc[try_1['owner'] == 0, 'year']
#try_1.loc[try_1['owner'] == 0, 'month']

#missing_owners = pd.DataFrame([try_1.loc[try_1['owner'] == 0, 'upc']], [try_1.loc[try_1['owner'] == 0, 'year']], [try_1.loc[try_1['owner'] == 0, 'month']])



In [784]:
upc_brand_entire_dates

empty_to_merge

,upc,year,month
0,1380014660,2012,5
1,1380014660,2012,6
2,1380014660,2012,7
3,1380014660,2012,8
4,1380014660,2012,9
...,...,...,...
723,74215878775,2016,4
724,74215878775,2016,5
725,74215878775,2016,6
726,74215878775,2016,7


In [740]:
df_own, brand_to_owner = append_owners(code, df, 'month',add_dhhi = False)

brand_to_owner[['start_month', 'start_year', 'end_month', 'end_year']]

,start_month,start_year,end_month,end_year
brand_code_uc,,,,
551720,5,2012,8,2016
567743,5,2012,8,2016
586896,5,2012,8,2016
586898,5,2012,8,2016
607137,5,2012,8,2016
658185,5,2012,8,2016
664037,5,2012,8,2016
669876,5,2012,8,2016


In [715]:
upc_brand_entire_dates.set_index('brand_code_uc').join(brand_to_owner.set_index('brand_code_uc'))

ValueError: columns overlap but no suffix specified: Index(['owner'], dtype='object')

In [684]:
info_dict = parse_info(code)
date_range = get_date_range(info_dict['DateAnnounced'], info_dict['DateCompleted'], pre_months = 24, post_months = 24)

date_range

,0,1
0,2012,05
1,2012,06
2,2012,07
3,2012,08
4,2012,09
5,2012,10
6,2012,11
7,2012,12
8,2013,01
9,2013,02


In [571]:
codes = '1924129020_1'

# data table
try_2 = vivek_table(code)

# empty table
info_dict = parse_info(code)
date_range = get_date_range(info_dict['DateAnnounced'], info_dict['DateCompleted'], pre_months = 24, post_months = 24)
    
# calculate number of rows there should be
upcs = (pd.read_csv('../../../../All/m_' + code + '/intermediate/upcs.csv', delimiter = ','))['upc']
    
total_rows = len(upcs)*len(date_range)
total_columns = len(try_2.columns)

empty_to_merge = pd.concat([repeated_upcs, repeated_dates], axis= 1)
empty_to_merge_full = pd.DataFrame(0, index = np.arange(total_rows), columns = try_1.columns[3:])
empty_to_merge_full.insert(0, 'upc', empty_to_merge['upc'])
empty_to_merge_full.insert(1, 'year', empty_to_merge['year'])
empty_to_merge_full.insert(2, 'month', empty_to_merge['month'])
empty_to_merge_full = empty_to_merge_full.apply(pd.to_numeric)

empty_to_merge_full.to_csv('try_2.csv', index = False, sep = ',', encoding = 'utf-8')

upc    year month owner sales                      ...  \
dma_code                                     500 501 503 504 505 506  ...   
0         1.380015e+09  2012.0   5.0     0     0   0   0   0   0   0  ...   
1         1.380015e+09  2012.0   6.0     0     0   0   0   0   0   0  ...   
2         1.380015e+09  2012.0   7.0     0     0   0   0   0   0   0  ...   
3         1.380015e+09  2012.0   8.0     0     0   0   0   0   0   0  ...   
4         1.380015e+09  2012.0   9.0     0     0   0   0   0   0   0  ...   
...                ...     ...   ...   ...   ...  ..  ..  ..  ..  ..  ...   
1097               NaN     NaN   NaN     0     0   0   0   0   0   0  ...   
1098               NaN     NaN   NaN     0     0   0   0   0   0   0  ...   
1099               NaN     NaN   NaN     0     0   0   0   0   0   0  ...   
1100               NaN     NaN   NaN     0     0   0   0   0   0   0  ...   
1101               NaN     NaN   NaN     0     0   0   0   0   0   0  ...   

         volume                 total_sales total_volume sold_in_usa  \
dma_code    855 862 866 868 881                                        
0             0   0   0   0   0           0            0           0   
1             0   0   0   0   0           0            0           0   
2             0   0   0   0   0           0            0           0   
3             0   0   0   0   0           0            0           0   
4             0   0   0   0   0           0            0           0   
...         ...  ..  ..  ..  ..         ...          ...         ...   
1097          0   0   0   0   0           0            0           0   
1098          0   0   0   0   0           0            0           0   
1099          0   0   0   0   0           0            0           0   
1100          0   0   0   0   0           0            0           0   
1101          0   0   0   0   0           0            0           0   

         merging_party post_merger  
dma_code                            
0                    0           0  
1                    0           0  
2                    0           0  
3                    0           0  
4                    0           0  
...                ...         ...  
1097                 0           0  
1098                 0           0  
1099                 0           0  
1100                 0           0  
1101                 0           0  

[1102 rows x 393 columns]

In [570]:
try_2.sort_values(by = ['upc'])

try_2.to_csv('try_2.csv', index = False, sep = ',', encoding = 'utf-8')

In [562]:
empty_to_merge_full = empty_to_merge_full.apply(pd.to_numeric)
empty_to_merge = empty_to_merge.apply(pd.to_numeric)

pd.merge(try_1, empty_to_merge_full, how = "right", on = ['upc', 'year', 'month'])



#on = ['upc', 'year', 'month'],  empty_to_merge_full.merge(try_1)


#empty_to_merge_full
try_1.sort_values(by = ['upc'])
empty_to_merge_full.sort_values(by = ['upc'])


upc  year month owner sales                      ... volume  \
dma_code                                  500 501 503 504 505 506  ...    855   
0          1380014660  2012     5     0     0   0   0   0   0   0  ...      0   
28         1380014660  2014     9     0     0   0   0   0   0   0  ...      0   
29         1380014660  2014    10     0     0   0   0   0   0   0  ...      0   
30         1380014660  2014    11     0     0   0   0   0   0   0  ...      0   
31         1380014660  2014    12     0     0   0   0   0   0   0  ...      0   
..                ...   ...   ...   ...   ...  ..  ..  ..  ..  ..  ...    ...   
696       74215878775  2014     1     0     0   0   0   0   0   0  ...      0   
697       74215878775  2014     2     0     0   0   0   0   0   0  ...      0   
698       74215878775  2014     3     0     0   0   0   0   0   0  ...      0   
700       74215878775  2014     5     0     0   0   0   0   0   0  ...      0   
727       74215878775  2016     8     0     0   0   0   0   0   0  ...      0   

                         total_sales total_volume sold_in_usa merging_party  \
dma_code 862 866 868 881                                                      
0          0   0   0   0           0            0           0             0   
28         0   0   0   0           0            0           0             0   
29         0   0   0   0           0            0           0             0   
30         0   0   0   0           0            0           0             0   
31         0   0   0   0           0            0           0             0   
..        ..  ..  ..  ..         ...          ...         ...           ...   
696        0   0   0   0           0            0           0             0   
697        0   0   0   0           0            0           0             0   
698        0   0   0   0           0            0           0             0   
700        0   0   0   0           0            0           0             0   
727        0   0   0   0           0            0           0             0   

         post_merger  
dma_code              
0                  0  
28                 0  
29                 0  
30                 0  
31                 0  
..               ...  
696                0  
697                0  
698                0  
700                0  
727                0  

[728 rows x 393 columns]

In [503]:
code = '1924129020_1'

try_1 = vivek_table(code)

len(try_1.columns)

empty_to_merge_full = pd.DataFrame(0, index = np.arange(total_rows), columns = try_1.columns[3:])

empty_to_merge = pd.concat([repeated_upcs, repeated_dates], axis= 1)

empty_to_merge_full.insert(0, 'upc', empty_to_merge['upc'])
empty_to_merge_full.insert(1, 'year', empty_to_merge['year'])
empty_to_merge_full.insert(2, 'month', empty_to_merge['month'])



In [471]:
empty_to_merge_1.merge(empty_to_merge, on = ['upc', 'year', 'month'], how = "inner")




ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [139]:
year = int((info_dict['DateCompleted'])[:4])
month = int((info_dict['DateCompleted'])[5:7])

In [140]:
print(month)

In [153]:
# create list of all year month combinations
info_dict['DateCompleted']

date_range = get_date_range(info_dict['DateAnnounced'], info_dict['DateCompleted'], pre_months = 24, post_months = 24)

type(date_range)
    
# add row of zeroes for all that are missing

# two ways - check if exists, if not add that row with zeros
# if combination of upc year month doesn't exist 
# add row


# way 2 - make entire table then merge

pandas.core.frame.DataFrame

In [210]:
date_range = get_date_range(info_dict['DateAnnounced'], info_dict['DateCompleted'], pre_months = 24, post_months = 24)
    
# calculate number of rows there should be
upcs = pd.read_csv('../../../../All/m_' + code + '/intermediate/upcs.csv', delimiter = ',')
total_rows = len(upcs)*len(date_range)

total_rows

728

In [415]:
# attempt as dataframe

upcs = (pd.read_csv('../../../../All/m_' + code + '/intermediate/upcs.csv', delimiter = ','))['upc']
date_range = get_date_range(info_dict['DateAnnounced'], info_dict['DateCompleted'], pre_months = 24, post_months = 24)

repeated_upcs = (pd.concat([upcs]*len(date_range))).sort_values(ignore_index = True)
repeated_upcs.columns = ['upc']

repeated_dates = pd.concat([date_range]*len(upcs), ignore_index = True, names = ['year', 'month'])
repeated_dates.columns = ['year', 'month']


empty_to_merge = pd.concat([repeated_upcs, repeated_dates], axis= 1)

empty_to_merge(index = np.arange(total_rows), columns = np.arange(len(try_1.columns)))


AttributeError: 'NoneType' object has no attribute 'columns'